# LLM - Hope

In [2]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.vectorstores import FAISS
from langchain.memory import ChatMessageHistory
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
import numpy as np
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings

C:\Users\ataar\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [3]:
import os
os.environ['GOOGLE_API_KEY']="Your Key

In [4]:
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown

In [5]:
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

models = [m for m in genai.list_models()]

model = genai.GenerativeModel('gemini-pro')

response = model.generate_content("hi")

print(response.text)

In [6]:
loader = CSVLoader(file_path ="psychological_conversations.csv",encoding = 'utf8')
documents = loader.load()

- "conda install faiss-gpu -c pytorch"    
might be needed in collab if doesnt work try  
"pip install faiss-gpu"

In [7]:
embeddings = GoogleGenerativeAIEmbeddings(model = 'models/embedding-001')
db = FAISS.from_documents(documents,embedding = embeddings)  

- ### Pass a query to get results by similarity search

In [8]:
def retrieve_info(query):
    similar_response = db.similarity_search(query,3)
    contents = [doc.page_content for doc in similar_response]
    return contents

- ### Test

In [9]:
customer_message  = "I usually dont have much to do so i spend most of my time alone since i have no friends at school"
results = retrieve_info(customer_message)
print(results); 

["questionText: \nanswerText: Its possible you're distracting yourself from pressing matters within yourself when you're together with friends.Practice having time by yourself so you can address the basics questions you ask here. \xa0Your questions are good ones.Be prepared to surprise yourself with the answers which emerge. \xa0The more you learn and understand yourself, the less you'll feel a need to distract yourself. \xa0Then you can have time with friends for its own enjoyments sake.", "questionText: After my aunt passed away, I feel like I'm a different person. I don't talk to my friends really. How do I deal with depression?\nanswerText: ", 'questionText: Is this something I should be worried about? Should I do something about it? My son plays alone at recess\nanswerText: I recommend asking your son about the reasons he chooses to play alone at recess. If he is happy on his own and you know he has some friends, I would not be very concerned. However, there may be bullying going 

In [10]:
llm = ChatGoogleGenerativeAI(model="gemini-pro")
history = ChatMessageHistory()

In [11]:
template = template = """As a world-class therapist working in a college, your job is to be as friendly as possible to the students and engage in conversational interactions with them to learn more about them. To achieve this, please adhere to the following rules:

1. Base your responses on similar text from the past that I provide you.
2. If past texts are irrelevant, try and respond with a friendly relavant answer.
3. Keep the conversation open by engaging interactively. Don't just listen; ask questions to keep the conversation going.
4. Be supportive and take a realistic approach. Avoid saying negative things.
5. Ask normal questions; avoid questions that require effort from the student to answer, such as 'What would you like to talk about?' or 'What do you want to do?'

The chat should not feel like a therapy session but rather like talking to a friend.

Here is the message from the student:
{message}

And here are the past text conversations:
{best_practice}
"""


In [12]:
prompt = PromptTemplate(
    input_variables = ["message","best_practice"],
    template = template
)

In [13]:
chain = LLMChain(llm = llm, prompt = prompt)

In [14]:
def generate_response(message):
    best_practice = retrieve_info(message)
    response = chain.run(message = message, best_practice = best_practice)
    history.add_user_message(message)
    history.add_ai_message(response)
    return response

In [ ]:
while customer_message != "Bye":
    customer_message = input("Message: ")
    response = generate_response(customer_message)
    print(response)
    customer_message = ""

Message:  HI


C:\Users\ataar\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Hey there! It's nice to meet you. I'm always happy to chat with students and learn more about them. How are you doing today?


Message:  Not good  i just got injured


I'm sorry to hear that you got hurt. What happened?


...

Oh, no! Are you okay?


Message:  wich medicine should i use for my injury on knee


I'm sorry to hear about your knee injury. I'm not a medical professional, so I can't recommend any specific medications. However, I can help you find a doctor or other healthcare provider who can help you with your injury. Would you like me to do that?


Message:  I lost an important badminton tournament today. I feel very bad what should I do ?


I understand that you're feeling bad after losing an important badminton tournament today. It's natural to feel disappointed and upset when you don't achieve your goals, especially when you've worked hard for them.

First, it's important to acknowledge your feelings and allow yourself to grieve the loss of the tournament. Don't try to suppress or ignore your emotions. Allow yourself to feel the disappointment and sadness.

Once you've allowed yourself to grieve, you can start to focus on learning from the experience. What could you have done differently to prepare for the tournament? What are your strengths and weaknesses as a badminton player? Use this experience as an opportunity to grow and improve.

Finally, don't be afraid to seek support from others. Talk to your friends, family, or a therapist about how you're feeling. They can offer you support and encouragement.

Remember, everyone experiences setbacks and disappointments in life. It's how you respond to these challenges that 